<center><h1>濒危大型动物种类识别挑战赛</h1></center>

> 报名链接：https://challenge.xfyun.cn/topic/info?type=species-recognition&option=ssgy&ch=dw24_AtTCK9


# 一、赛事背景

全球生物多样性持续下降，尤其濒危大型动物的数量锐减，成为全球环保的紧急议题。濒危动物如大象、犀牛及大型猫科动物面临众多威胁，如栖息地丧失、非法狩猎等，这不仅威胁到生物多样性，也影响生态平衡。为此，精确及时的监测和保护措施显得至关重要。然而，传统监测方法耗时且效率低，难以满足迫切需求。借助人工智能技术，开发自动化的濒危大型动物种类识别系统，通过分析图像数据自动识别特定动物，将极大提升保护效率和准确性。因此，举办一场濒危大型动物种类识别挑战赛，具有重要的研究意义，旨在推动AI技术在野生动物保护中的应用，提高公众保护意识，促进全球生态保护工作。

# 二、赛事任务

本次濒危大型动物种类识别挑战赛旨在应用人工智能技术，提高对濒危大型动物的监测与保护效率。赛事提供了包括照相陷阱、无人机等多源采集的动物图像数据及其所在保护区的地理位置信息。参赛者需基于这些数据，开发出能够准确识别濒危大型动物种类的模型。挑战的关键在于处理和识别在不同光照、角度和背景条件下相似物种。

# 三、评审规则

## 1.数据说明

本次濒危大型动物种类识别挑战赛为参赛选手提供了一个包含9种不同濒危动物的庞大图像数据库，这些图像通过照相陷阱、无人机等多样化的采集方法，在不同的时间、光照条件和多样化的自然背景下收集，旨在模拟实际野外监测的多种情形。

（1）数据集细分说明：

训练集用于模型训练，提供动物图像及其对应的类别标签；测试集用于评估模型性能，仅包含图像数据，参赛者需要预测图像对应的动物种类。

（2）种类识别数据集具体分类：

| Label       |
| ----------- |
| Badger      |
| BlackBear   |
| Cheetah     |
| Hare        |
| LeopardCat  |
| MuskDeer    |
| AmurLeopard |
| Tiger       |
| RedFox      |

## 2.评估指标

本模型依据提交的结果文件，采用macro-F1分数进行评价。

## 3.评测及排行

（1）初赛和复赛均提供下载数据，选手在本地进行算法调试，在比赛页面提交结果。

（2）比赛采用AB榜，A榜成绩供参赛队伍比赛中查看，最终比赛排名采用B榜最佳成绩。

# 四、作品提交要求

1、文件格式：按照csv格式提交

2、文件大小：无要求

3、提交次数限制：每支队伍每天最多3次

4、文件详细说明：编码为UTF-8，第一行为表头，提交格式见样例

5、不需要上传其他文件

# 五、赛程规则

本赛题实行一轮赛制

## 【赛程周期】

7月17日-8月29日

1、7月17日10：00发布训练集、开发集、即开启比赛榜单，8月15日发布B榜测试

2、比赛作品提交截止日期为8月29日17：00，公布名次日期为9月9日10:00

## 【现场答辩】

1、最终前三名团队将受邀参加科大讯飞全球1024开发者节并于现场进行答辩

2、答辩以（10mins陈述+5mins问答）的形式进行

3、根据作品成绩和答辩成绩综合评分（作品成绩占比70％，现场答辩分数占比30％）

# 六、奖项设置

本赛题设立一、二、三等奖共三名，具体详情如下：

## 【奖项激励】

1.  TOP3团队颁发获奖证书
2.  赛道奖金，第一名5000元、第二名3000元、第三名2000元

## 【资源激励】

1.  讯飞开放平台优质AI能力个人资源包
2.  讯飞AI全链创业扶持资源
3.  讯飞绿色实习/就业通道

注：

1.  鼓励选手分享参赛心得、参赛技术攻略、大赛相关技术或产品使用体验等文章至组委会邮箱（AICompetition@iflytek.com），有机会获得大赛周边；
2.  赛事规则及奖金发放解释权归科大讯飞所有；以上全部奖金均为税前金额，将由主办方代扣代缴个人所得税。

In [4]:
!pip install timm pandas

Looking in indexes: http://mirrors.aliyun.com/pypi/simple
     |████████████████████████████████| 12.4 MB 561 kB/s eta 0:00:01
     |████████████████████████████████| 345 kB 550 kB/s eta 0:00:01


In [1]:
import torch
torch.manual_seed(0)
torch.backends.cudnn.deterministic = False
torch.backends.cudnn.benchmark = True

import torchvision.models as models
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data.dataset import Dataset
import time
import glob

import pandas as pd
import numpy as np
from PIL import Image
from tqdm import tqdm_notebook

In [2]:
label_list = ['Badger', 'BlackBear', 'Cheetah', 'Hare', 'LeopardCat', 'MuskDeer', 'AmurLeopard', 'Tiger', 'RedFox']

train_path = glob.glob('./train/*/*.jpg')
np.random.shuffle(train_path)
train_label = [label_list.index(x.split('/')[-2]) for x in train_path]

test_path = glob.glob('./testA/*.jpg')

In [3]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self, name, fmt=':f'):
        self.name = name
        self.fmt = fmt
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

    def __str__(self):
        fmtstr = '{name} {val' + self.fmt + '} ({avg' + self.fmt + '})'
        return fmtstr.format(**self.__dict__)

class ProgressMeter(object):
    def __init__(self, num_batches, *meters):
        self.batch_fmtstr = self._get_batch_fmtstr(num_batches)
        self.meters = meters
        self.prefix = ""


    def pr2int(self, batch):
        entries = [self.prefix + self.batch_fmtstr.format(batch)]
        entries += [str(meter) for meter in self.meters]
        print('\t'.join(entries))

    def _get_batch_fmtstr(self, num_batches):
        num_digits = len(str(num_batches // 1))
        fmt = '{:' + str(num_digits) + 'd}'
        return '[' + fmt + '/' + fmt.format(num_batches) + ']'
def validate(val_loader, model, criterion):
    batch_time = AverageMeter('Time', ':6.3f')
    losses = AverageMeter('Loss', ':.4e')
    top1 = AverageMeter('Acc@1', ':6.2f')
    progress = ProgressMeter(len(val_loader), batch_time, losses, top1)

    # switch to evaluate mode
    model.eval()

    with torch.no_grad():
        end = time.time()
        for i, (input, target) in tqdm_notebook(enumerate(val_loader), total=len(val_loader)):
            input = input.cuda()
            target = target.cuda()

            # compute output
            output = model(input)
            loss = criterion(output, target)

            # measure accuracy and record loss
            acc = (output.argmax(1).view(-1) == target.float().view(-1)).float().mean() * 100
            losses.update(loss.item(), input.size(0))
            top1.update(acc, input.size(0))
            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()

        # TODO: this should also be done with the ProgressMeter
        print(' * Acc@1 {top1.avg:.3f}'
              .format(top1=top1))
        return top1

def predict(test_loader, model, tta=10):
    # switch to evaluate mode
    model.eval()
    
    test_pred_tta = None
    for _ in range(tta):
        test_pred = []
        with torch.no_grad():
            end = time.time()
            for i, (input, target) in tqdm_notebook(enumerate(test_loader), total=len(test_loader)):
                input = input.cuda()
                target = target.cuda()

                # compute output
                output = model(input)
                output = F.softmax(output, dim=1)
                output = output.data.cpu().numpy()

                test_pred.append(output)
        test_pred = np.vstack(test_pred)
    
        if test_pred_tta is None:
            test_pred_tta = test_pred
        else:
            test_pred_tta += test_pred
    
    return test_pred_tta

def train(train_loader, model, criterion, optimizer, epoch):
    batch_time = AverageMeter('Time', ':6.3f')
    losses = AverageMeter('Loss', ':.4e')
    top1 = AverageMeter('Acc@1', ':6.2f')
    progress = ProgressMeter(len(train_loader), batch_time, losses, top1)

    # switch to train mode
    model.train()

    end = time.time()
    for i, (input, target) in enumerate(train_loader):
        input = input.cuda(non_blocking=True)
        target = target.cuda(non_blocking=True)

        # compute output
        output = model(input)
        loss = criterion(output, target)

        # measure accuracy and record loss
        losses.update(loss.item(), input.size(0))

        acc = (output.argmax(1).view(-1) == target.float().view(-1)).float().mean() * 100
        top1.update(acc, input.size(0))

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        if i % 100 == 0:
            progress.pr2int(i)

In [4]:
class XFDataset(Dataset):
    def __init__(self, img_path, img_label, transform=None):
        self.img_path = img_path
        self.img_label = img_label
        
        if transform is not None:
            self.transform = transform
        else:
            self.transform = None
    
    def __getitem__(self, index):
        img = Image.open(self.img_path[index]).convert('RGB')
        
        if self.transform is not None:
            img = self.transform(img)
        
        return img, torch.from_numpy(np.array(self.img_label[index]))
    
    def __len__(self):
        return len(self.img_path)

In [13]:
import os
os.environ['HF_ENDPOINT'] = 'https://hf-mirror.com'

import timm
model = timm.create_model('efficientnet_b1', pretrained=False, num_classes=9)
model = model.cuda()

In [14]:
train_loader = torch.utils.data.DataLoader(
    XFDataset(train_path[:-500], train_label[:-500], 
            transforms.Compose([
                        transforms.Resize((256, 256)),
                        transforms.RandomHorizontalFlip(),
                        transforms.RandomVerticalFlip(),
                        transforms.ColorJitter(brightness=.5, hue=.3),
                        transforms.ToTensor(),
                        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])
    ), batch_size=70, shuffle=True, num_workers=4, pin_memory=True
)

val_loader = torch.utils.data.DataLoader(
    XFDataset(train_path[-500:], train_label[-500:], 
            transforms.Compose([
                        transforms.Resize((256, 256)),
                        transforms.ToTensor(),
                        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])
    ), batch_size=70, shuffle=False, num_workers=4, pin_memory=True
)

criterion = nn.CrossEntropyLoss().cuda()
optimizer = torch.optim.Adam(model.parameters(), 0.007)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=4, gamma=0.85)
best_acc = 0.0
for epoch in range(20):
    scheduler.step()
    print('Epoch: ', epoch)

    train(train_loader, model, criterion, optimizer, epoch)
    val_acc = validate(val_loader, model, criterion)
    
    if val_acc.avg.item() > best_acc:
        best_acc = round(val_acc.avg.item(), 2)
        torch.save(model.state_dict(), f'./model_{best_acc}.pt')

Epoch:  0


/root/miniconda3/lib/python3.8/site-packages/torch/optim/lr_scheduler.py:129: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "


[ 0/96]	Time  2.672 ( 2.672)	Loss 4.4935e+00 (4.4935e+00)	Acc@1  10.00 ( 10.00)


/tmp/ipykernel_1641/2676678360.py:51: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i, (input, target) in tqdm_notebook(enumerate(val_loader), total=len(val_loader)):


  0%|          | 0/8 [00:00<?, ?it/s]

...

  0%|          | 0/8 [00:00<?, ?it/s]

 * Acc@1 89.800
Epoch:  6
[ 0/96]	Time  2.137 ( 2.137)	Loss 2.1807e-01 (2.1807e-01)	Acc@1  91.43 ( 91.43)


  0%|          | 0/8 [00:00<?, ?it/s]

 * Acc@1 88.600
Epoch:  7
[ 0/96]	Time  2.508 ( 2.508)	Loss 3.7349e-01 (3.7349e-01)	Acc@1  88.57 ( 88.57)


KeyboardInterrupt: 

In [15]:
test_loader = torch.utils.data.DataLoader(
    XFDataset(test_path, [0] * len(test_path), 
            transforms.Compose([
                        transforms.Resize((256, 256)),
                        transforms.ToTensor(),
                        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])
    ), batch_size=40, shuffle=False, num_workers=4, pin_memory=True
)

val_label = pd.DataFrame()
val_label['uuid'] = [x.split('/')[-1] for x in test_path]
val_label['y_pred'] = predict(test_loader, model, 1).argmax(1)
val_label['label'] = val_label['y_pred'].apply(lambda x: label_list[x])
val_label[['uuid', 'label']].to_csv('submit.csv', index=None)

/tmp/ipykernel_1641/2676678360.py:81: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i, (input, target) in tqdm_notebook(enumerate(test_loader), total=len(test_loader)):


  0%|          | 0/60 [00:00<?, ?it/s]

0            RedFox
1            Badger
2            RedFox
3        LeopardCat
4       AmurLeopard
           ...     
2377           Hare
2378    AmurLeopard
2379           Hare
2380         Badger
2381     LeopardCat
Name: y_pred, Length: 2382, dtype: object